## Method
- Make prediction for city level using experimental design 2, Naive Counting and SEPPexp based on week-days model.
- Compare model performance using hit rate traditional implementation, true positives proportion and PAI.

## Hypothesis:
SEPP exp prediction based on week-days performance is better than the obtained with counting model.

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import sys
sys.path.append("..")
from services.prediction_experiment import PredictionExperiment

Failed to import `rtree`.
Failed to import `rtree`.


In [3]:
from services import prediction_metrics

In [4]:
def get_hit_rate_from_dict(row,column,coverage):
    return row[column][coverage]

## SIEDCO

In [5]:
csv_path = '/Users/anamaria/Desktop/dev/security_project/datasets/deduplicate_siedco_09062020.csv'
siedco_info = {'name':'SIEDCO','path':csv_path}

In [6]:
train_dates = {'initial':'2018-03-01','final':'2018-09-30'}
validation_dates = {'initial':'2018-10-01','final':'2018-10-07'}
metrics = ''
aggregation = ''

## Counting

In [7]:
filter_localidad = {'field':'','value':''}
model = "NaiveCounting"
localidad_experiment = PredictionExperiment(dataset_info=siedco_info, custom_filter=filter_localidad,train_dates=train_dates, validation_dates=validation_dates, model=model,metrics='',aggregation_data='')
prediction_array = localidad_experiment.run_ncv_experiment(time_unit='',grid_size=150, outer_iterations='')

/Users/anamaria/Desktop/dev/security_project/security_venv/lib/python3.7/site-packages/ipykernel_launcher.py:4: DtypeWarning: Columns (32,44) have mixed types. Specify dtype option on import or set low_memory=False.
  after removing the cwd from sys.path.


train_subset_dates {'initial': datetime.datetime(2018, 3, 1, 0, 0), 'final': datetime.datetime(2018, 9, 30, 0, 0)}
current_validation_date 2018-10-01 00:00:00
14073
train_subset_dates {'initial': datetime.datetime(2018, 3, 1, 0, 0), 'final': datetime.datetime(2018, 10, 1, 0, 0)}
current_validation_date 2018-10-02 00:00:00
14114
train_subset_dates {'initial': datetime.datetime(2018, 3, 1, 0, 0), 'final': datetime.datetime(2018, 10, 2, 0, 0)}
current_validation_date 2018-10-03 00:00:00
14156
train_subset_dates {'initial': datetime.datetime(2018, 3, 1, 0, 0), 'final': datetime.datetime(2018, 10, 3, 0, 0)}
current_validation_date 2018-10-04 00:00:00
14218
train_subset_dates {'initial': datetime.datetime(2018, 3, 1, 0, 0), 'final': datetime.datetime(2018, 10, 4, 0, 0)}
current_validation_date 2018-10-05 00:00:00
14281
train_subset_dates {'initial': datetime.datetime(2018, 3, 1, 0, 0), 'final': datetime.datetime(2018, 10, 5, 0, 0)}
current_validation_date 2018-10-06 00:00:00
14344
train_subs

In [10]:
df_counting = pd.DataFrame(prediction_array, columns =['initial-date','final-date','prediction','eval_pts'])
df_result = df_counting.copy()

In [ ]:
coverages = [2,4,6,8,10,12,14,16,18,20]
df_result['hitrate_default'] = df_result.apply(lambda row: prediction_metrics.measure_hit_rates(row['prediction'],row['eval_pts'],coverages,'default'), axis=1)
df_result['hitrate_TP'] = df_result.apply(lambda row: prediction_metrics.measure_hit_rates(row['prediction'],row['eval_pts'],coverages,'ground_truth_coverage'), axis=1)


In [23]:
##For hitrate_TP "true positives"
df_result['coverage_TP'] = df_result.apply(lambda row: list(row['hitrate_TP'].keys())[0], axis=1)
df_result['hit_rate_TP'] = df_result.apply(lambda row: list(row['hitrate_TP'].values())[0], axis=1)
df_result['PAI_TP'] = df_result['hit_rate_TP'] / (df_result['coverage_TP']/100)

In [24]:
##For hitrate_default
coverages = [2.0,4.0,6.0,8.0,10.0,12.0,14.0,16.0,18.0,20.0]
column_dict = 'hitrate_default'
for c in coverages:
    new_hit_rate_column = 'hit_rate_default_coverage_'+str(c)
    df_result[new_hit_rate_column] = df_result.apply(lambda row: get_hit_rate_from_dict(row,column_dict,c), axis=1)

    ##PAI
    new_column = 'PAI_default_coverage_'+str(c)
    df_result[new_column] = df_result[new_hit_rate_column]/(c/100)


In [25]:
#delete datetimes where no crimes were reported (0 crimes on ground truth -> hit-rate = -1)
df_result = df_result[df_result['hit_rate_default_coverage_2.0']!= -1]
print(df_result.mean())

coverage_TP                         0.008963
hit_rate_TP                         0.046744
PAI_TP                            630.987368
hit_rate_default_coverage_2.0       0.493017
PAI_default_coverage_2.0           24.650830
hit_rate_default_coverage_4.0       0.681974
PAI_default_coverage_4.0           17.049347
hit_rate_default_coverage_6.0       0.731925
PAI_default_coverage_6.0           12.198750
hit_rate_default_coverage_8.0       0.731925
PAI_default_coverage_8.0            9.149062
hit_rate_default_coverage_10.0      0.731925
PAI_default_coverage_10.0           7.319250
hit_rate_default_coverage_12.0      0.739706
PAI_default_coverage_12.0           6.164216
hit_rate_default_coverage_14.0      0.764172
PAI_default_coverage_14.0           5.458369
hit_rate_default_coverage_16.0      0.786687
PAI_default_coverage_16.0           4.916795
hit_rate_default_coverage_18.0      0.794150
PAI_default_coverage_18.0           4.411946
hit_rate_default_coverage_20.0      0.807006
PAI_defaul

In [26]:
df_result

,initial-date,final-date,prediction,eval_pts,hitrate_default,hitrate_TP,coverage_TP,hit_rate_TP,PAI_TP,hit_rate_default_coverage_2.0,...,hit_rate_default_coverage_12.0,PAI_default_coverage_12.0,hit_rate_default_coverage_14.0,PAI_default_coverage_14.0,hit_rate_default_coverage_16.0,PAI_default_coverage_16.0,hit_rate_default_coverage_18.0,PAI_default_coverage_18.0,hit_rate_default_coverage_20.0,PAI_default_coverage_20.0
0,2018-10-01 00:00:00,2018-10-01 06:00:00,"GridPredictionArray(offset=(971268.1246075855,...",<open_cp.data.TimedPoints object at 0x12822c128>,"{2: 0.5555555555555556, 4: 0.6666666666666666,...",{0.005523336095001381: 0.0},0.005523,0.000000,0.000000,0.555556,...,0.666667,5.555556,0.666667,4.761905,0.666667,4.166667,0.666667,3.703704,0.666667,3.333333
1,2018-10-01 06:00:00,2018-10-01 12:00:00,"GridPredictionArray(offset=(971268.1246075855,...",<open_cp.data.TimedPoints object at 0x12aa30ac8>,"{2: 0.75, 4: 1.0, 6: 1.0, 8: 1.0, 10: 1.0, 12:...",{0.0049096320844456715: 0.0},0.004910,0.000000,0.000000,0.750000,...,1.000000,8.333333,1.000000,7.142857,1.000000,6.250000,1.000000,5.555556,1.000000,5.000000
2,2018-10-01 12:00:00,2018-10-01 18:00:00,"GridPredictionArray(offset=(971268.1246075855,...",<open_cp.data.TimedPoints object at 0x12aa30a90>,"{2: 0.3, 4: 0.5, 6: 0.5, 8: 0.5, 10: 0.5, 12: ...",{0.00613704010555709: 0.1},0.006137,0.100000,1629.450000,0.300000,...,0.500000,4.166667,0.500000,3.571429,0.600000,3.750000,0.600000,3.333333,0.600000,3.000000
3,2018-10-01 18:00:00,2018-10-02 00:00:00,"GridPredictionArray(offset=(971268.1246075855,...",<open_cp.data.TimedPoints object at 0x12aa30978>,"{2: 0.6428571428571429, 4: 0.6428571428571429,...",{0.008591856147779926: 0.0},0.008592,0.000000,0.000000,0.642857,...,0.785714,6.547619,0.857143,6.122449,0.857143,5.357143,0.857143,4.761905,0.857143,4.285714
4,2018-10-02 00:00:00,2018-10-02 06:00:00,"GridPredictionArray(offset=(971268.1246075855,...",<open_cp.data.TimedPoints object at 0x12aa30d30>,"{2: 0.7142857142857143, 4: 0.8571428571428571,...",{0.004295928073889963: 0.14285714285714285},0.004296,0.142857,3325.408163,0.714286,...,0.857143,7.142857,0.857143,6.122449,0.857143,5.357143,0.857143,4.761905,0.857143,4.285714
5,2018-10-02 06:00:00,2018-10-02 12:00:00,"GridPredictionArray(offset=(971268.1246075855,...",<open_cp.data.TimedPoints object at 0x12aa30c18>,"{2: 0.4375, 4: 0.75, 6: 0.8125, 8: 0.8125, 10:...",{0.009819264168891343: 0.0},0.009819,0.000000,0.000000,0.437500,...,0.812500,6.770833,0.812500,5.803571,0.812500,5.078125,0.812500,4.513889,0.812500,4.062500
6,2018-10-02 12:00:00,2018-10-02 18:00:00,"GridPredictionArray(offset=(971268.1246075855,...",<open_cp.data.TimedPoints object at 0x12aa30898>,"{2: 0.4, 4: 0.4, 6: 0.4, 8: 0.4, 10: 0.4, 12: ...",{0.003068520052778545: 0.0},0.003069,0.000000,0.000000,0.400000,...,0.400000,3.333333,0.600000,4.285714,1.000000,6.250000,1.000000,5.555556,1.000000,5.000000
7,2018-10-02 18:00:00,2018-10-03 00:00:00,"GridPredictionArray(offset=(971268.1246075855,...",<open_cp.data.TimedPoints object at 0x12aa30828>,"{2: 0.5714285714285714, 4: 0.7142857142857143,...",{0.008591856147779926: 0.0},0.008592,0.000000,0.000000,0.571429,...,0.785714,6.547619,0.785714,5.612245,0.785714,4.910714,0.785714,4.365079,0.785714,3.928571
8,2018-10-03 00:00:00,2018-10-03 06:00:00,"GridPredictionArray(offset=(971268.1246075855,...",<open_cp.data.TimedPoints object at 0x12aa30e80>,"{2: 0.625, 4: 0.75, 6: 0.75, 8: 0.75, 10: 0.75...",{0.009819264168891343: 0.1875},0.009819,0.187500,1909.511719,0.625000,...,0.750000,6.250000,0.750000,5.357143,0.750000,4.687500,0.750000,4.166667,0.750000,3.750000
9,2018-10-03 06:00:00,2018-10-03 12:00:00,"GridPredictionArray(offset=(971268.1246075855,...",<open_cp.data.TimedPoints object at 0x12aa30780>,"{2: 0.18181818181818182, 4: 0.3636363636363636...",{0.0067507441161127986: 0.0},0.006751,0.000000,0.000000,0.181818,...,0.636364,5.303030,0.636364,4.545455,0.636364,3.977273,0.636364,3.535354,0.636364,3.181818


## Counting (reduce base dates)
This approach is proposed to keep a similar number of training points (considering this model 'NaiveCounting' and background kernel on sepp the prediction is related to the historical spatial distribution/concentration of events) 

In [12]:
train_dates = {'initial':'2018-09-01','final':'2018-09-30'}
validation_dates = {'initial':'2018-10-01','final':'2018-10-07'}
metrics = ''
aggregation = ''

In [13]:
filter_localidad = {'field':'','value':''}
model = "NaiveCounting"
localidad_experiment = PredictionExperiment(dataset_info=siedco_info, custom_filter=filter_localidad,train_dates=train_dates, validation_dates=validation_dates, model=model,metrics='',aggregation_data='')
prediction_array = localidad_experiment.run_ncv_experiment(time_unit='',grid_size=150, outer_iterations='')

/Users/anamaria/Desktop/dev/security_project/security_venv/lib/python3.7/site-packages/ipykernel_launcher.py:4: DtypeWarning: Columns (32,44) have mixed types. Specify dtype option on import or set low_memory=False.
  after removing the cwd from sys.path.


train_subset_dates {'initial': datetime.datetime(2018, 9, 1, 0, 0), 'final': datetime.datetime(2018, 9, 30, 0, 0)}
current_validation_date 2018-10-01 00:00:00
1940
train_subset_dates {'initial': datetime.datetime(2018, 9, 1, 0, 0), 'final': datetime.datetime(2018, 10, 1, 0, 0)}
current_validation_date 2018-10-02 00:00:00
1981
train_subset_dates {'initial': datetime.datetime(2018, 9, 1, 0, 0), 'final': datetime.datetime(2018, 10, 2, 0, 0)}
current_validation_date 2018-10-03 00:00:00
2023
train_subset_dates {'initial': datetime.datetime(2018, 9, 1, 0, 0), 'final': datetime.datetime(2018, 10, 3, 0, 0)}
current_validation_date 2018-10-04 00:00:00
2085
train_subset_dates {'initial': datetime.datetime(2018, 9, 1, 0, 0), 'final': datetime.datetime(2018, 10, 4, 0, 0)}
current_validation_date 2018-10-05 00:00:00
2148
train_subset_dates {'initial': datetime.datetime(2018, 9, 1, 0, 0), 'final': datetime.datetime(2018, 10, 5, 0, 0)}
current_validation_date 2018-10-06 00:00:00
2211
train_subset_dat

In [14]:
df_counting_2 = pd.DataFrame(prediction_array, columns =['initial-date','final-date','prediction','eval_pts'])
df_result = df_counting_2.copy()

In [ ]:
coverages = [2,4,6,8,10,12,14,16,18,20]
df_result['hitrate_default'] = df_result.apply(lambda row: prediction_metrics.measure_hit_rates(row['prediction'],row['eval_pts'],coverages,'default'), axis=1)
df_result['hitrate_TP'] = df_result.apply(lambda row: prediction_metrics.measure_hit_rates(row['prediction'],row['eval_pts'],coverages,'ground_truth_coverage'), axis=1)


In [15]:
##For hitrate_TP "true positives"
df_result['coverage_TP'] = df_result.apply(lambda row: list(row['hitrate_TP'].keys())[0], axis=1)
df_result['hit_rate_TP'] = df_result.apply(lambda row: list(row['hitrate_TP'].values())[0], axis=1)
df_result['PAI_TP'] = df_result['hit_rate_TP'] / (df_result['coverage_TP']/100)

In [16]:
##For hitrate_default
coverages = [2.0,4.0,6.0,8.0,10.0,12.0,14.0,16.0,18.0,20.0]
column_dict = 'hitrate_default'
for c in coverages:
    new_hit_rate_column = 'hit_rate_default_coverage_'+str(c)
    df_result[new_hit_rate_column] = df_result.apply(lambda row: get_hit_rate_from_dict(row,column_dict,c), axis=1)

    ##PAI
    new_column = 'PAI_default_coverage_'+str(c)
    df_result[new_column] = df_result[new_hit_rate_column]/(c/100)


In [17]:
#delete datetimes where no crimes were reported (0 crimes on ground truth -> hit-rate = -1)
df_result = df_result[df_result['hit_rate_default_coverage_2.0']!= -1]
print(df_result.mean())

coverage_TP                         0.033395
hit_rate_TP                         0.037881
PAI_TP                            120.801055
hit_rate_default_coverage_2.0       0.148028
PAI_default_coverage_2.0            7.401406
hit_rate_default_coverage_4.0       0.280370
PAI_default_coverage_4.0            7.009256
hit_rate_default_coverage_6.0       0.322161
PAI_default_coverage_6.0            5.369358
hit_rate_default_coverage_8.0       0.327611
PAI_default_coverage_8.0            4.095142
hit_rate_default_coverage_10.0      0.332270
PAI_default_coverage_10.0           3.322698
hit_rate_default_coverage_12.0      0.334614
PAI_default_coverage_12.0           2.788451
hit_rate_default_coverage_14.0      0.334614
PAI_default_coverage_14.0           2.390101
hit_rate_default_coverage_16.0      0.345095
PAI_default_coverage_16.0           2.156846
hit_rate_default_coverage_18.0      0.353220
PAI_default_coverage_18.0           1.962336
hit_rate_default_coverage_20.0      0.361811
PAI_defaul

In [18]:
df_result

,initial-date,final-date,prediction,eval_pts,hitrate_default,hitrate_TP,coverage_TP,hit_rate_TP,PAI_TP,hit_rate_default_coverage_2.0,...,hit_rate_default_coverage_12.0,PAI_default_coverage_12.0,hit_rate_default_coverage_14.0,PAI_default_coverage_14.0,hit_rate_default_coverage_16.0,PAI_default_coverage_16.0,hit_rate_default_coverage_18.0,PAI_default_coverage_18.0,hit_rate_default_coverage_20.0,PAI_default_coverage_20.0
0,2018-10-01 00:00:00,2018-10-01 06:00:00,"GridPredictionArray(offset=(985137.4254012154,...",<open_cp.data.TimedPoints object at 0x124e8d668>,"{2: 0.2222222222222222, 4: 0.4444444444444444,...",{0.02452048823016565: 0.0},0.024520,0.000000,0.000000,0.222222,...,0.444444,3.703704,0.444444,3.174603,0.444444,2.777778,0.444444,2.469136,0.444444,2.222222
1,2018-10-01 06:00:00,2018-10-01 12:00:00,"GridPredictionArray(offset=(985137.4254012154,...",<open_cp.data.TimedPoints object at 0x124e8d5c0>,"{2: 0.125, 4: 0.375, 6: 0.375, 8: 0.375, 10: 0...",{0.021795989537925022: 0.0},0.021796,0.000000,0.000000,0.125000,...,0.375000,3.125000,0.375000,2.678571,0.375000,2.343750,0.375000,2.083333,0.375000,1.875000
2,2018-10-01 12:00:00,2018-10-01 18:00:00,"GridPredictionArray(offset=(985137.4254012154,...",<open_cp.data.TimedPoints object at 0x124e8d518>,"{2: 0.2, 4: 0.4, 6: 0.4, 8: 0.4, 10: 0.4, 12: ...",{0.02724498692240628: 0.1},0.027245,0.100000,367.040000,0.200000,...,0.400000,3.333333,0.400000,2.857143,0.400000,2.500000,0.400000,2.222222,0.400000,2.000000
3,2018-10-01 18:00:00,2018-10-02 00:00:00,"GridPredictionArray(offset=(985137.4254012154,...",<open_cp.data.TimedPoints object at 0x124e8d438>,"{2: 0.0, 4: 0.07142857142857142, 6: 0.14285714...",{0.03814298169136879: 0.0},0.038143,0.000000,0.000000,0.000000,...,0.142857,1.190476,0.142857,1.020408,0.142857,0.892857,0.214286,1.190476,0.214286,1.071429
4,2018-10-02 00:00:00,2018-10-02 06:00:00,"GridPredictionArray(offset=(985137.4254012154,...",<open_cp.data.TimedPoints object at 0x124e8d8d0>,"{2: 0.2857142857142857, 4: 0.7142857142857143,...",{0.019071490845684394: 0.14285714285714285},0.019071,0.142857,749.061224,0.285714,...,0.714286,5.952381,0.714286,5.102041,0.714286,4.464286,0.714286,3.968254,0.714286,3.571429
5,2018-10-02 06:00:00,2018-10-02 12:00:00,"GridPredictionArray(offset=(985137.4254012154,...",<open_cp.data.TimedPoints object at 0x124e8d780>,"{2: 0.0, 4: 0.0, 6: 0.0625, 8: 0.0625, 10: 0.0...",{0.043591979075850044: 0.0},0.043592,0.000000,0.000000,0.000000,...,0.062500,0.520833,0.062500,0.446429,0.062500,0.390625,0.062500,0.347222,0.062500,0.312500
6,2018-10-02 12:00:00,2018-10-02 18:00:00,"GridPredictionArray(offset=(985137.4254012154,...",<open_cp.data.TimedPoints object at 0x124e8d6d8>,"{2: 0.0, 4: 0.2, 6: 0.2, 8: 0.2, 10: 0.2, 12: ...",{0.01362249346120314: 0.0},0.013622,0.000000,0.000000,0.000000,...,0.200000,1.666667,0.200000,1.428571,0.200000,1.250000,0.200000,1.111111,0.200000,1.000000
7,2018-10-02 18:00:00,2018-10-03 00:00:00,"GridPredictionArray(offset=(985137.4254012154,...",<open_cp.data.TimedPoints object at 0x124e8da90>,"{2: 0.14285714285714285, 4: 0.2142857142857142...",{0.03814298169136879: 0.07142857142857142},0.038143,0.071429,187.265306,0.142857,...,0.285714,2.380952,0.285714,2.040816,0.285714,1.785714,0.285714,1.587302,0.357143,1.785714
8,2018-10-03 00:00:00,2018-10-03 06:00:00,"GridPredictionArray(offset=(985137.4254012154,...",<open_cp.data.TimedPoints object at 0x124e8da58>,"{2: 0.3125, 4: 0.375, 6: 0.375, 8: 0.375, 10: ...",{0.043591979075850044: 0.125},0.043592,0.125000,286.750000,0.312500,...,0.375000,3.125000,0.375000,2.678571,0.375000,2.343750,0.375000,2.083333,0.375000,1.875000
9,2018-10-03 06:00:00,2018-10-03 12:00:00,"GridPredictionArray(offset=(985137.4254012154,...",<open_cp.data.TimedPoints object at 0x124e8d358>,"{2: 0.09090909090909091, 4: 0.1818181818181818...",{0.029969485614646903: 0.0},0.029969,0.000000,0.000000,0.090909,...,0.272727,2.272727,0.272727,1.948052,0.363636,2.272727,0.363636,2.020202,0.363636,1.818182


## SEPPexp week-day

In [27]:
filter_localidad = {'field':'','value':''}
model = "SEPPexpWeekDay"
localidad_experiment = PredictionExperiment(dataset_info=siedco_info, custom_filter=filter_localidad,train_dates=train_dates, validation_dates=validation_dates, model=model,metrics='',aggregation_data='')
prediction_array = localidad_experiment.run_ncv_experiment(time_unit='',grid_size=150, outer_iterations='')

/Users/anamaria/Desktop/dev/security_project/security_venv/lib/python3.7/site-packages/ipykernel_launcher.py:4: DtypeWarning: Columns (32,44) have mixed types. Specify dtype option on import or set low_memory=False.
  after removing the cwd from sys.path.


train_subset_dates {'initial': datetime.datetime(2018, 3, 1, 0, 0), 'final': datetime.datetime(2018, 9, 30, 0, 0)}
current_validation_date 2018-10-01 00:00:00


../services/aggressive_model.py:54: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_train_subset['weekday'] = df_train_subset['TIME_STAMP'].dt.day_name()


omega: 0.06644245864035621, theta: 0.013628663475551774
1714
train_subset_dates {'initial': datetime.datetime(2018, 3, 1, 0, 0), 'final': datetime.datetime(2018, 10, 1, 0, 0)}
current_validation_date 2018-10-02 00:00:00
omega: 0.023284820274164272, theta: 0.013892301755967921
1741
train_subset_dates {'initial': datetime.datetime(2018, 3, 1, 0, 0), 'final': datetime.datetime(2018, 10, 2, 0, 0)}
current_validation_date 2018-10-03 00:00:00
omega: 0.11381653749534727, theta: 0.008902223096009859
1658
train_subset_dates {'initial': datetime.datetime(2018, 3, 1, 0, 0), 'final': datetime.datetime(2018, 10, 3, 0, 0)}
current_validation_date 2018-10-04 00:00:00
omega: 0.1260160450913719, theta: 0.01002400513607306
1765
train_subset_dates {'initial': datetime.datetime(2018, 3, 1, 0, 0), 'final': datetime.datetime(2018, 10, 4, 0, 0)}
current_validation_date 2018-10-05 00:00:00
omega: 0.05617779775387991, theta: 0.012004555441094692
1856
train_subset_dates {'initial': datetime.datetime(2018, 3, 1,

In [28]:
df_seppexp = pd.DataFrame(prediction_array, columns =['initial-date','final-date','prediction','eval_pts'])
df_result = df_seppexp.copy()

In [29]:
coverages = [2,4,6,8,10,12,14,16,18,20]
df_result['hitrate_default'] = df_result.apply(lambda row: prediction_metrics.measure_hit_rates(row['prediction'],row['eval_pts'],coverages,'default'), axis=1)
df_result['hitrate_TP'] = df_result.apply(lambda row: prediction_metrics.measure_hit_rates(row['prediction'],row['eval_pts'],coverages,'ground_truth_coverage'), axis=1)


In [30]:
##For hitrate_TP "true positives"
df_result['coverage_TP'] = df_result.apply(lambda row: list(row['hitrate_TP'].keys())[0], axis=1)
df_result['hit_rate_TP'] = df_result.apply(lambda row: list(row['hitrate_TP'].values())[0], axis=1)
df_result['PAI_TP'] = df_result['hit_rate_TP'] / (df_result['coverage_TP']/100)

In [31]:
##For hitrate_default
coverages = [2.0,4.0,6.0,8.0,10.0,12.0,14.0,16.0,18.0,20.0]
column_dict = 'hitrate_default'
for c in coverages:
    new_hit_rate_column = 'hit_rate_default_coverage_'+str(c)
    df_result[new_hit_rate_column] = df_result.apply(lambda row: get_hit_rate_from_dict(row,column_dict,c), axis=1)

    ##PAI
    new_column = 'PAI_default_coverage_'+str(c)
    df_result[new_column] = df_result[new_hit_rate_column]/(c/100)


In [32]:
#delete datetimes where no crimes were reported (0 crimes on ground truth -> hit-rate = -1)
df_result = df_result[df_result['hit_rate_default_coverage_2.0']!= -1]
print(df_result.mean())

coverage_TP                         0.032343
hit_rate_TP                         0.038442
PAI_TP                            290.094757
hit_rate_default_coverage_2.0       0.144308
PAI_default_coverage_2.0            7.215418
hit_rate_default_coverage_4.0       0.208397
PAI_default_coverage_4.0            5.209936
hit_rate_default_coverage_6.0       0.248920
PAI_default_coverage_6.0            4.148670
hit_rate_default_coverage_8.0       0.262052
PAI_default_coverage_8.0            3.275652
hit_rate_default_coverage_10.0      0.279319
PAI_default_coverage_10.0           2.793192
hit_rate_default_coverage_12.0      0.290638
PAI_default_coverage_12.0           2.421983
hit_rate_default_coverage_14.0      0.295313
PAI_default_coverage_14.0           2.109380
hit_rate_default_coverage_16.0      0.306646
PAI_default_coverage_16.0           1.916539
hit_rate_default_coverage_18.0      0.326740
PAI_default_coverage_18.0           1.815225
hit_rate_default_coverage_20.0      0.379406
PAI_defaul

In [33]:
df_result

,initial-date,final-date,prediction,eval_pts,hitrate_default,hitrate_TP,coverage_TP,hit_rate_TP,PAI_TP,hit_rate_default_coverage_2.0,...,hit_rate_default_coverage_12.0,PAI_default_coverage_12.0,hit_rate_default_coverage_14.0,PAI_default_coverage_14.0,hit_rate_default_coverage_16.0,PAI_default_coverage_16.0,hit_rate_default_coverage_18.0,PAI_default_coverage_18.0,hit_rate_default_coverage_20.0,PAI_default_coverage_20.0
0,2018-10-01 00:00:00,2018-10-01 06:00:00,"GridPredictionArray(offset=(984795.4834218245,...",<open_cp.data.TimedPoints object at 0x1285b2780>,"{2: 0.1111111111111111, 4: 0.2222222222222222,...",{0.024454528163464936: 0.0},0.024455,0.000000,0.000000,0.111111,...,0.222222,1.851852,0.222222,1.587302,0.222222,1.388889,0.222222,1.234568,0.222222,1.111111
1,2018-10-01 06:00:00,2018-10-01 12:00:00,"GridPredictionArray(offset=(984795.4834218245,...",<open_cp.data.TimedPoints object at 0x1233144a8>,"{2: 0.25, 4: 0.25, 6: 0.25, 8: 0.25, 10: 0.25,...",{0.021737358367524388: 0.0},0.021737,0.000000,0.000000,0.250000,...,0.250000,2.083333,0.250000,1.785714,0.250000,1.562500,0.250000,1.388889,0.250000,1.250000
2,2018-10-01 12:00:00,2018-10-01 18:00:00,"GridPredictionArray(offset=(984795.4834218245,...",<open_cp.data.TimedPoints object at 0x12885fe80>,"{2: 0.2, 4: 0.2, 6: 0.2, 8: 0.2, 10: 0.2, 12: ...",{0.027171697959405484: 0.0},0.027172,0.000000,0.000000,0.200000,...,0.200000,1.666667,0.200000,1.428571,0.200000,1.250000,0.200000,1.111111,0.200000,1.000000
3,2018-10-01 18:00:00,2018-10-02 00:00:00,"GridPredictionArray(offset=(984795.4834218245,...",<open_cp.data.TimedPoints object at 0x12b7ea208>,"{2: 0.0, 4: 0.21428571428571427, 6: 0.21428571...",{0.03804037714316768: 0.0},0.038040,0.000000,0.000000,0.000000,...,0.214286,1.785714,0.214286,1.530612,0.214286,1.339286,0.285714,1.587302,0.357143,1.785714
4,2018-10-02 00:00:00,2018-10-02 06:00:00,"GridPredictionArray(offset=(973660.0863242749,...",<open_cp.data.TimedPoints object at 0x12a484c18>,"{2: 0.2857142857142857, 4: 0.2857142857142857,...",{0.0056874502348104455: 0.14285714285714285},0.005687,0.142857,2511.795918,0.285714,...,0.285714,2.380952,0.285714,2.040816,0.285714,1.785714,0.285714,1.587302,0.428571,2.142857
5,2018-10-02 06:00:00,2018-10-02 12:00:00,"GridPredictionArray(offset=(973660.0863242749,...",<open_cp.data.TimedPoints object at 0x1266386d8>,"{2: 0.125, 4: 0.125, 6: 0.125, 8: 0.1875, 10: ...",{0.012999886250995303: 0.0},0.013000,0.000000,0.000000,0.125000,...,0.187500,1.562500,0.187500,1.339286,0.187500,1.171875,0.187500,1.041667,0.437500,2.187500
6,2018-10-02 12:00:00,2018-10-02 18:00:00,"GridPredictionArray(offset=(973660.0863242749,...",<open_cp.data.TimedPoints object at 0x1266384e0>,"{2: 0.2, 4: 0.2, 6: 0.2, 8: 0.2, 10: 0.6, 12: ...",{0.004062464453436033: 0.0},0.004062,0.000000,0.000000,0.200000,...,0.800000,6.666667,0.800000,5.714286,0.800000,5.000000,0.800000,4.444444,0.800000,4.000000
7,2018-10-02 18:00:00,2018-10-03 00:00:00,"GridPredictionArray(offset=(973660.0863242749,...",<open_cp.data.TimedPoints object at 0x12885fdd8>,"{2: 0.07142857142857142, 4: 0.0714285714285714...",{0.011374900469620891: 0.0},0.011375,0.000000,0.000000,0.071429,...,0.285714,2.380952,0.285714,2.040816,0.285714,1.785714,0.500000,2.777778,0.500000,2.500000
8,2018-10-03 00:00:00,2018-10-03 06:00:00,"GridPredictionArray(offset=(985326.2999210021,...",<open_cp.data.TimedPoints object at 0x1266388d0>,"{2: 0.3125, 4: 0.3125, 6: 0.3125, 8: 0.3125, 1...",{0.047619047619047616: 0.1875},0.047619,0.187500,393.750000,0.312500,...,0.312500,2.604167,0.312500,2.232143,0.312500,1.953125,0.312500,1.736111,0.312500,1.562500
9,2018-10-03 06:00:00,2018-10-03 12:00:00,"GridPredictionArray(offset=(985326.2999210021,...",<open_cp.data.TimedPoints object at 0x126610278>,"{2: 0.0, 4: 0.09090909090909091, 6: 0.18181818...",{0.03273809523809524: 0.0},0.032738,0.000000,0.000000,0.000000,...,0.272727,2.272727,0.272727,1.948052,0.272727,1.704545,0.272727,1.515152,0.272727,1.363636
